In [1]:
import pandas as pd
import numpy as np
import os

# Hàm split_dataframe giữ nguyên.
def split_dataframe(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Nhận một DataFrame, tách nó thành hai phần (đúng và cần crawl lại)
    dựa trên các điều kiện đã xác định và trả về hai DataFrame đã được
    loại bỏ trùng lặp.
    """
    df_copy = df.copy()

    main_link_correct = df_copy['MainLinkCorrect'].fillna('').astype(str).str.strip().str.upper()
    date_correct = df_copy['ConferenceDateCorrect'].fillna('').astype(str).str.strip().str.upper()
    location_correct = df_copy['LocationCorrect'].fillna('').astype(str).str.strip().str.upper()
    type_correct = df_copy['TypeCorrect'].fillna('').astype(str).str.strip().str.upper()

    condition1 = (
        (main_link_correct == 'FALSE') |
        (date_correct == 'FALSE') |
        (location_correct == 'FALSE') |
        (type_correct == 'FALSE')

    )

    percent_cols = ['Percent', 'Percent_1', 'Percent_2', 'Percent_3', 'Percent_4']
    for col in percent_cols:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

    condition2_parts = [df_copy[col].notna() & (df_copy[col] != 100) for col in percent_cols]
    condition2 = False
    for part in condition2_parts:
        condition2 = condition2 | part

    recrawl_condition = condition1 | condition2
    recrawl_df = df_copy[recrawl_condition]
    correct_df = df_copy[~recrawl_condition]

    subset_cols = ['title', 'acronym']
    unique_recrawl_df = recrawl_df.drop_duplicates(subset=subset_cols, keep='first')
    unique_correct_df = correct_df.drop_duplicates(subset=subset_cols, keep='first')

    return unique_correct_df, unique_recrawl_df


def process_file_list(
    input_files: list, 
    output_dir: str,
    aggregated_correct_path: str, 
    aggregated_recrawl_path: str,
    recrawled_files: list = None
):
    """
    Xử lý một danh sách các file CSV, đối chiếu với kết quả đã recrawl,
    và tạo ra hai file tổng hợp cuối cùng.
    """
    all_correct_dfs = []
    all_recrawl_dfs = []

    os.makedirs(output_dir, exist_ok=True)

    # --- BƯỚC 1: XỬ LÝ CÁC FILE INPUT BAN ĐẦU ---
    for file_path in input_files:
        print(f"\n--- Đang xử lý file input: {file_path} ---")
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
            print(f"Đã đọc thành công {len(df)} dòng.")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Đã xảy ra lỗi khi đọc file '{file_path}': {e}. Bỏ qua file này.")
            continue

        correct_df, recrawl_df = split_dataframe(df)
        all_correct_dfs.append(correct_df)
        all_recrawl_dfs.append(recrawl_df)

        # Lưu file riêng lẻ (tùy chọn)
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        individual_correct_path = os.path.join(output_dir, f"{base_name}_correct.csv")
        individual_recrawl_path = os.path.join(output_dir, f"{base_name}_recrawl.csv")
        correct_df.to_csv(individual_correct_path, index=False, encoding='utf-8-sig')
        recrawl_df.to_csv(individual_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu output riêng lẻ vào thư mục '{output_dir}'")

    # --- BƯỚC 2: GỘP KẾT QUẢ TỪ CÁC FILE INPUT BAN ĐẦU ---
    if not all_correct_dfs and not all_recrawl_dfs:
        print("\nKhông có dữ liệu từ các file input để xử lý. Kết thúc.")
        return

    print("\n--- Đang tổng hợp kết quả từ các file input ---")
    master_correct_df = pd.concat(all_correct_dfs, ignore_index=True) if all_correct_dfs else pd.DataFrame()
    master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True) if all_recrawl_dfs else pd.DataFrame()

    # --- BƯỚC 3: ĐỐI CHIẾU VỚI CÁC FILE ĐÃ RECRAWL ---
    if recrawled_files:
        print("\n--- Đang đối chiếu với kết quả đã recrawl ---")
        recrawled_dfs_list = []
        for r_file in recrawled_files:
            try:
                df_recrawled = pd.read_csv(r_file, encoding='utf-8-sig', na_values=['', ' '], dtype=str)
                recrawled_dfs_list.append(df_recrawled)
                print(f"Đã đọc file đã recrawl: {r_file}")
            except FileNotFoundError:
                print(f"Cảnh báo: Không tìm thấy file đã recrawl '{r_file}'. Bỏ qua.")
            except Exception as e:
                print(f"Lỗi khi đọc file đã recrawl '{r_file}': {e}. Bỏ qua.")
        
        if recrawled_dfs_list:
            recrawled_master_df = pd.concat(recrawled_dfs_list, ignore_index=True)
            
            # Dùng lại hàm split_dataframe để xác định những dòng nào giờ đã ĐÚNG theo tiêu chí chính
            newly_correct_df_from_recrawl, _ = split_dataframe(recrawled_master_df)
            
            if not newly_correct_df_from_recrawl.empty:
                print(f"Tìm thấy {len(newly_correct_df_from_recrawl)} dòng đã được sửa đúng sau khi recrawl (theo tiêu chí Correct/Percent).")
                
                # Tạo set các khóa (title, acronym) của các dòng đã được sửa đúng
                # Đảm bảo title/acronym không trống trước khi tạo keys
                newly_correct_df_from_recrawl['title'] = newly_correct_df_from_recrawl['title'].str.strip()
                if 'acronym' in newly_correct_df_from_recrawl.columns:
                    newly_correct_df_from_recrawl['acronym'] = newly_correct_df_from_recrawl['acronym'].str.strip()
                
                subset_keys_dedup = ['title']
                if 'acronym' in newly_correct_df_from_recrawl.columns:
                    subset_keys_dedup.append('acronym')
                
                # Filter out rows with empty title/acronym after strip
                newly_correct_df_from_recrawl = newly_correct_df_from_recrawl[newly_correct_df_from_recrawl['title'] != '']
                if 'acronym' in newly_correct_df_from_recrawl.columns:
                    newly_correct_df_from_recrawl = newly_correct_df_from_recrawl[newly_correct_df_from_recrawl['acronym'] != '']


                corrected_keys = set(tuple(x) for x in newly_correct_df_from_recrawl[subset_keys_dedup].values)
                
                # Loại bỏ các dòng đã được sửa đúng khỏi danh sách recrawl tổng ban đầu
                # (Những dòng này đã được xử lý và giờ đã đúng theo tiêu chí chính)
                master_recrawl_df['title_stripped'] = master_recrawl_df['title'].str.strip()
                master_recrawl_df['acronym_stripped'] = master_recrawl_df['acronym'].str.strip()
                
                recrawl_keys_to_check = set(tuple(x) for x in master_recrawl_df[['title_stripped', 'acronym_stripped']].values)

                # Identify which keys from master_recrawl_df are now in corrected_keys
                keys_to_move_from_recrawl = recrawl_keys_to_check.intersection(corrected_keys)

                # Create a mask for rows to be removed from master_recrawl_df
                mask_to_remove = master_recrawl_df[['title_stripped', 'acronym_stripped']].apply(tuple, axis=1).isin(keys_to_move_from_recrawl)
                
                rows_moved_to_correct = master_recrawl_df[mask_to_remove].copy()
                master_recrawl_df = master_recrawl_df[~mask_to_remove].copy()

                # Thêm các dòng vừa được sửa đúng vào master_correct_df
                master_correct_df = pd.concat([master_correct_df, newly_correct_df_from_recrawl], ignore_index=True)
                
                print(f"Đã di chuyển {len(rows_moved_to_correct)} dòng từ recrawl tổng sang correct tổng (sau đối chiếu).")
            else:
                print("Không có dòng nào được sửa đúng trong các file đã recrawl (theo tiêu chí Correct/Percent).")
        else:
            print("Không có file đã recrawl nào hợp lệ để đối chiếu.")
    
    # Clean up temporary columns
    if 'title_stripped' in master_correct_df.columns:
        master_correct_df.drop(columns=['title_stripped', 'acronym_stripped'], inplace=True)
    if 'title_stripped' in master_recrawl_df.columns:
        master_recrawl_df.drop(columns=['title_stripped', 'acronym_stripped'], inplace=True)


    # --- BƯỚC 4 (QUAN TRỌNG NHẤT): LỌC CUỐI CÙNG DỰA TRÊN CỘT 'Note' ---
    print("\n--- Lọc cuối cùng: Đảm bảo không có dòng nào có 'Note' trong danh sách đúng ---")
    if 'Note' in master_correct_df.columns:
        # Điều kiện: cột 'Note' có giá trị (không phải NaN hoặc rỗng)
        note_exists_condition_final = master_correct_df['Note'].notna() & (master_correct_df['Note'].astype(str).str.strip() != '')
        
        # Lấy ra những dòng có ghi chú từ DataFrame 'correct'
        rows_to_move_final = master_correct_df[note_exists_condition_final].copy() # .copy() để tránh SettingWithCopyWarning
        
        if not rows_to_move_final.empty:
            print(f"Tìm thấy {len(rows_to_move_final)} dòng trong dữ liệu 'đúng' VẪN CÒN ghi chú. Di chuyển chúng sang danh sách 'cần recrawl'.")
            
            # 1. Thêm những dòng này vào DataFrame 'recrawl'
            master_recrawl_df = pd.concat([master_recrawl_df, rows_to_move_final], ignore_index=True)
            
            # 2. Loại bỏ những dòng này khỏi DataFrame 'correct'
            master_correct_df = master_correct_df[~note_exists_condition_final].copy() # .copy()
        else:
            print("Không có dòng nào trong dữ liệu 'đúng' có ghi chú sau các bước lọc. Đã đạt yêu cầu.")
    else:
        print("Cảnh báo: Không tìm thấy cột 'Note' trong dữ liệu tổng. Bỏ qua bước kiểm tra ghi chú cuối cùng.")


    # --- BƯỚC 5: LOẠI BỎ TRÙNG LẶP LẦN CUỐI VÀ LƯU FILE ---
    print("\n--- Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Xử lý và lưu file CORRECT tổng
    if not master_correct_df.empty:
        # Đảm bảo các cột dùng để drop_duplicates không bị NaN/rỗng
        master_correct_df['title'] = master_correct_df['title'].astype(str).str.strip()
        
        subset_for_dedup_correct = ['title']
        if 'acronym' in master_correct_df.columns:
            master_correct_df['acronym'] = master_correct_df['acronym'].astype(str).str.strip()
            subset_for_dedup_correct.append('acronym')
        
        master_correct_df = master_correct_df[master_correct_df['title'] != '']
        if 'acronym' in master_correct_df.columns:
             master_correct_df = master_correct_df[master_correct_df['acronym'] != '']
        
        final_correct_df = master_correct_df.drop_duplicates(subset=subset_for_dedup_correct, keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng đúng vào '{aggregated_correct_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào. Đã tạo file rỗng '{aggregated_correct_path}'")

    # Xử lý và lưu file RECRAWL tổng
    if not master_recrawl_df.empty:
        # Đảm bảo các cột dùng để drop_duplicates không bị NaN/rỗng
        master_recrawl_df['title'] = master_recrawl_df['title'].astype(str).str.strip()
        
        subset_for_dedup_recrawl = ['title']
        if 'acronym' in master_recrawl_df.columns:
            master_recrawl_df['acronym'] = master_recrawl_df['acronym'].astype(str).str.strip()
            subset_for_dedup_recrawl.append('acronym')

        master_recrawl_df = master_recrawl_df[master_recrawl_df['title'] != '']
        if 'acronym' in master_recrawl_df.columns:
             master_recrawl_df = master_recrawl_df[master_recrawl_df['acronym'] != '']

        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=subset_for_dedup_recrawl, keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần crawl lại vào '{aggregated_recrawl_path}'")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input ban đầu
    input_csv_files = [
        './src/conference/evaluate/batch2.csv',
        './src/conference/evaluate/batch3.csv',
        './src/conference/evaluate/batch8.csv',
        './src/conference/evaluate/batch12.csv',
        './src/conference/evaluate/batch13.csv',
        './src/conference/evaluate/batch16.csv',
        './src/conference/evaluate/batch19.csv',
    ]

    # 2. Danh sách các file là kết quả của lần recrawl trước đó
    recrawled_results_files = [
        './src/conference/evaluate/evaluate_recrawl_batch_2_3_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_8_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_12_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_13_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_16_lan_1.csv',
        './src/conference/evaluate/evaluate_recrawl_batch_19_lan_1.csv',
        
        # THÊM CÁC FILE RECRAWL MỚI VÀO ĐÂY
        './src/conference/evaluate/evaluate_recrawl_all_tung_1_50.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_51_100.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_101_150.csv',
        './src/conference/evaluate/evaluate_recrawl_all_tung_151_159.csv',

    ]

    # 3. Thư mục để chứa các file output riêng lẻ
    individual_output_directory = './src/conference/evaluate/individual_outputs'

    # 4. Đường dẫn cho 2 file tổng hợp cuối cùng
    aggregated_correct_file = './src/conference/evaluate/ALL_BATCHES_correct_final.csv'
    aggregated_recrawl_file = './src/conference/evaluate/ALL_BATCHES_recrawl_final.csv'

    # 5. Gọi hàm xử lý chính
    process_file_list(
        input_files=input_csv_files,
        output_dir=individual_output_directory,
        aggregated_correct_path=aggregated_correct_file,
        aggregated_recrawl_path=aggregated_recrawl_file,
        recrawled_files=recrawled_results_files
    )


--- Đang xử lý file input: ./src/conference/evaluate/batch2.csv ---
Đã đọc thành công 47 dòng.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/individual_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch3.csv ---
Đã đọc thành công 46 dòng.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/individual_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch8.csv ---
Đã đọc thành công 48 dòng.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/individual_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch12.csv ---
Đã đọc thành công 38 dòng.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/individual_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch13.csv ---
Đã đọc thành công 46 dòng.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/individual_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch16.csv ---
Đã đọc thành công 46 dòng.
Đã lưu outp

In [1]:
import pandas as pd
import os

def split_df_by_note(df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Tách một DataFrame thành hai phần dựa trên sự tồn tại của giá trị trong cột 'Note'.

    - Dữ liệu "cần recrawl": Các dòng có giá trị trong cột 'Note'.
    - Dữ liệu "đúng": Các dòng không có giá trị trong cột 'Note' (trống hoặc NaN).

    Args:
        df (pd.DataFrame): DataFrame đầu vào.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: Một tuple chứa (correct_df, recrawl_df).
    """
    # Kiểm tra an toàn để đảm bảo cột 'Note' tồn tại
    # Lưu ý: Sửa 'Note' thành 'note' nếu tên cột thực sự là chữ thường
    if 'note' not in df.columns: 
        print("Cảnh báo: Không tìm thấy cột 'note'. Coi như tất cả các dòng đều 'đúng'.")
        return df.copy(), pd.DataFrame(columns=df.columns)

    # Điều kiện: cột 'note' có giá trị (không phải NaN và không phải chuỗi rỗng sau khi strip)
    # Áp dụng .str.strip() trước khi so sánh
    condition_recrawl = df['note'].notna() & (df['note'].astype(str).str.strip() != '')

    recrawl_df = df[condition_recrawl].copy()
    correct_df = df[~condition_recrawl].copy()

    return correct_df, recrawl_df

def process_files_by_note(
    input_files: list, 
    output_dir: str,
    aggregated_correct_path: str, 
    aggregated_recrawl_path: str,
    recrawled_files: list = None # THÊM THAM SỐ MỚI
):
    """
    Xử lý một danh sách file, lọc từng file dựa trên cột 'Note'
    và chỉ xét những dòng có cột 'title' có giá trị.
    Sau đó đối chiếu với các file đã recrawl để loại bỏ những dòng đã được sửa đúng.
    Lưu output riêng lẻ và hai file tổng hợp cuối cùng.
    """
    all_correct_dfs = []
    all_recrawl_dfs = []

    os.makedirs(output_dir, exist_ok=True)

    # --- BƯỚC 1: XỬ LÝ CÁC FILE INPUT BAN ĐẦU ---
    for file_path in input_files:
        print(f"\n--- Đang xử lý file input: {file_path} ---")
        try:
            # Đọc file với dtype=str để đảm bảo cột 'note' và 'title' được đọc đúng dạng chuỗi
            df = pd.read_csv(file_path, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
            print(f"Đã đọc thành công {len(df)} dòng.")
        except FileNotFoundError:
            print(f"Lỗi: Không tìm thấy file '{file_path}'. Bỏ qua file này.")
            continue
        except Exception as e:
            print(f"Đã xảy ra lỗi khi đọc file '{file_path}': {e}. Bỏ qua file này.")
            continue

        # Chỉ xét những dòng có cột 'title' có giá trị
        if 'title' in df.columns:
            initial_rows = len(df)
            df = df[df['title'].notna() & (df['title'].astype(str).str.strip() != '')].copy()
            if len(df) < initial_rows:
                print(f"Đã loại bỏ {initial_rows - len(df)} dòng do cột 'title' trống.")
        else:
            print("Cảnh báo: Không tìm thấy cột 'title'. Không thể lọc theo tiêu chí 'title có giá trị'.")
        
        if df.empty:
            print("Không còn dòng nào sau khi lọc theo 'title'. Bỏ qua file này.")
            continue

        correct_df, recrawl_df = split_df_by_note(df)
        
        print(f"Kết quả tách ban đầu: {len(correct_df)} dòng không có ghi chú, {len(recrawl_df)} dòng có ghi chú.")

        if not correct_df.empty:
            all_correct_dfs.append(correct_df)
        if not recrawl_df.empty:
            all_recrawl_dfs.append(recrawl_df)

        # --- Lưu các file output riêng lẻ ---
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        individual_correct_path = os.path.join(output_dir, f"{base_name}_note_correct.csv")
        individual_recrawl_path = os.path.join(output_dir, f"{base_name}_note_recrawl.csv")

        correct_df.to_csv(individual_correct_path, index=False, encoding='utf-8-sig')
        recrawl_df.to_csv(individual_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu output riêng lẻ vào thư mục '{output_dir}'")

    # --- BƯỚC 2: GỘP CÁC DATAFRAME BAN ĐẦU ---
    if not all_correct_dfs and not all_recrawl_dfs:
        print("\nKhông có dữ liệu từ các file input để xử lý. Kết thúc.")
        return

    print("\n--- Đang tổng hợp kết quả từ các file input ---")
    master_correct_df = pd.concat(all_correct_dfs, ignore_index=True) if all_correct_dfs else pd.DataFrame()
    master_recrawl_df = pd.concat(all_recrawl_dfs, ignore_index=True) if all_recrawl_dfs else pd.DataFrame()

    # --- BƯỚC 3 (MỚI): ĐỐI CHIẾU VỚI KẾT QUẢ ĐÃ RECRAWL DỰA TRÊN 'note' ---
    if recrawled_files:
        print("\n--- Đang đối chiếu với kết quả đã recrawl (dựa trên cột 'note') ---")
        recrawled_dfs_list = []
        for r_file in recrawled_files:
            try:
                # Đọc file đã recrawl, đảm bảo dtype=str cho 'note' và 'title'
                df = pd.read_csv(r_file, encoding='utf-8-sig', na_values=[''], dtype=str).fillna('')
                # Lọc title cho file đã recrawl cũng
                if 'title' in df.columns:
                    df = df[df['title'].notna() & (df['title'].astype(str).str.strip() != '')].copy()
                else:
                    print(f"Cảnh báo: File đã recrawl '{r_file}' không có cột 'title'.")

                if not df.empty:
                    recrawled_dfs_list.append(df)
                    print(f"Đã đọc file đã recrawl: {r_file}")
                else:
                    print(f"File đã recrawl '{r_file}' trống sau khi lọc 'title'.")
            except FileNotFoundError:
                print(f"Cảnh báo: Không tìm thấy file đã recrawl '{r_file}'. Bỏ qua.")
            except Exception as e:
                print(f"Lỗi khi đọc file đã recrawl '{r_file}': {e}. Bỏ qua.")
        
        if recrawled_dfs_list:
            recrawled_master_df = pd.concat(recrawled_dfs_list, ignore_index=True)
            
            # Tái sử dụng split_df_by_note để xác định những dòng nào giờ đã "đúng" (không có note)
            newly_correct_after_recrawl_df, _ = split_df_by_note(recrawled_master_df)
            
            if not newly_correct_after_recrawl_df.empty:
                print(f"Tìm thấy {len(newly_correct_after_recrawl_df)} dòng đã không còn ghi chú sau khi recrawl.")
                
                # Chuẩn bị keys để đối chiếu
                # Đảm bảo title/acronym không trống trước khi tạo keys
                newly_correct_after_recrawl_df['title'] = newly_correct_after_recrawl_df['title'].str.strip()
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    newly_correct_after_recrawl_df['acronym'] = newly_correct_after_recrawl_df['acronym'].str.strip()

                subset_keys = ['title']
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    subset_keys.append('acronym')

                # Loại bỏ các dòng có title/acronym rỗng sau khi strip trước khi tạo keys
                newly_correct_after_recrawl_df = newly_correct_after_recrawl_df[newly_correct_after_recrawl_df['title'] != '']
                if 'acronym' in newly_correct_after_recrawl_df.columns:
                    newly_correct_after_recrawl_df = newly_correct_after_recrawl_df[newly_correct_after_recrawl_df['acronym'] != '']

                # Tạo set các khóa của những dòng đã được sửa đúng
                corrected_keys_after_recrawl = set(
                    tuple(x) for x in newly_correct_after_recrawl_df[subset_keys].values
                )
                
                # --- CẬP NHẬT master_recrawl_df và master_correct_df ---
                # 1. Loại bỏ các dòng đã được sửa đúng khỏi danh sách recrawl tổng
                # Tạo một Series với các khóa (title, acronym) của master_recrawl_df để so sánh
                master_recrawl_keys = master_recrawl_df[subset_keys].apply(tuple, axis=1)
                is_now_correct_mask = master_recrawl_keys.isin(corrected_keys_after_recrawl)
                
                # Giữ lại những dòng KHÔNG có trong danh sách đã sửa đúng
                master_recrawl_df_kept = master_recrawl_df[~is_now_correct_mask].copy()
                master_recrawl_df_moved = master_recrawl_df[is_now_correct_mask].copy() # Các dòng bị di chuyển
                
                master_recrawl_df = master_recrawl_df_kept
                print(f"Đã loại bỏ {len(master_recrawl_df_moved)} dòng khỏi danh sách recrawl tổng vì đã được sửa đúng.")

                # 2. Thêm các dòng vừa được sửa đúng vào danh sách correct tổng
                master_correct_df = pd.concat([master_correct_df, newly_correct_after_recrawl_df], ignore_index=True)
                print(f"Đã thêm {len(newly_correct_after_recrawl_df)} dòng vào danh sách đúng tổng.")
            else:
                print("Không có dòng nào được coi là 'đúng' trong các file đã recrawl (theo cột 'note').")
        else:
            print("Không có file đã recrawl nào hợp lệ để đối chiếu.")

    # --- BƯỚC 4: LOẠI BỎ TRÙNG LẶP LẦN CUỐI VÀ LƯU FILE ---
    print("\n--- Hoàn tất và lưu file tổng hợp cuối cùng ---")
    
    # Chuẩn bị cột cho drop_duplicates lần cuối
    subset_for_dedup = ['title']
    
    # Xử lý và lưu file CORRECT tổng
    if not master_correct_df.empty:
        # Chuẩn hóa title/acronym và loại bỏ các dòng rỗng trước khi final dedup
        master_correct_df['title'] = master_correct_df['title'].astype(str).str.strip()
        if 'acronym' in master_correct_df.columns:
            master_correct_df['acronym'] = master_correct_df['acronym'].astype(str).str.strip()
            subset_for_dedup_correct = ['title', 'acronym']
        else:
            subset_for_dedup_correct = ['title']

        master_correct_df = master_correct_df[master_correct_df['title'] != '']
        if 'acronym' in master_correct_df.columns:
             master_correct_df = master_correct_df[master_correct_df['acronym'] != '']

        final_correct_df = master_correct_df.drop_duplicates(subset=subset_for_dedup_correct, keep='first')
        final_correct_df.to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_correct_df)} dòng đúng (không có ghi chú) vào '{aggregated_correct_path}'.")
    else:
        pd.DataFrame().to_csv(aggregated_correct_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'đúng' nào. Đã tạo file rỗng '{aggregated_correct_path}'.")

    # Xử lý và lưu file RECRAWL tổng
    if not master_recrawl_df.empty:
        # Chuẩn hóa title/acronym và loại bỏ các dòng rỗng trước khi final dedup
        master_recrawl_df['title'] = master_recrawl_df['title'].astype(str).str.strip()
        if 'acronym' in master_recrawl_df.columns:
            master_recrawl_df['acronym'] = master_recrawl_df['acronym'].astype(str).str.strip()
            subset_for_dedup_recrawl = ['title', 'acronym']
        else:
            subset_for_dedup_recrawl = ['title']

        master_recrawl_df = master_recrawl_df[master_recrawl_df['title'] != '']
        if 'acronym' in master_recrawl_df.columns:
             master_recrawl_df = master_recrawl_df[master_recrawl_df['acronym'] != '']
             
        final_recrawl_df = master_recrawl_df.drop_duplicates(subset=subset_for_dedup_recrawl, keep='first')
        final_recrawl_df.to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Đã lưu tổng cộng {len(final_recrawl_df)} dòng cần recrawl (có ghi chú) vào '{aggregated_recrawl_path}'.")
    else:
        pd.DataFrame().to_csv(aggregated_recrawl_path, index=False, encoding='utf-8-sig')
        print(f"Không có dòng 'cần recrawl' nào. Đã tạo file rỗng '{aggregated_recrawl_path}'.")


# --- Cách sử dụng ---
if __name__ == "__main__":
    # 1. Danh sách các file input ban đầu (ví dụ: các file batch_X_check.csv)
    input_csv_files_for_note_check = [
        './src/conference/evaluate/batch4.csv',
        './src/conference/evaluate/batch5.csv',
        './src/conference/evaluate/batch9.csv',
        './src/conference/evaluate/batch14.csv',
        './src/conference/evaluate/batch15.csv',
        './src/conference/evaluate/batch18.csv',
        './src/conference/evaluate/batch20.csv',
    ]

    # 2. Danh sách các file là KẾT QUẢ của lần recrawl trước đó (sau khi đã được sửa)
    # Ví dụ: nếu bạn đã chạy recrawl cho batch4 và nhận được file recrawl_batch_4_result.csv
    recrawled_results_files = [
        './src/conference/evaluate/recrawl_batch_4_check.csv',
        './src/conference/evaluate/recrawl_batch_5_check.csv',
        './src/conference/evaluate/recrawl_batch_9_check.csv',
        './src/conference/evaluate/recrawl_batch_14_check.csv',
        './src/conference/evaluate/recrawl_batch_15_check.csv',
        './src/conference/evaluate/recrawl_batch_18_check.csv',
        './src/conference/evaluate/recrawl_batch_20_check.csv',

    ]

    # 3. Thư mục để chứa các file output riêng lẻ
    individual_output_directory_note = './src/conference/evaluate/tri_check_outputs'

    # 4. Đường dẫn cho 2 file tổng hợp cuối cùng
    aggregated_correct_file_note = './src/conference/evaluate/tri_all_correct_final.csv'
    aggregated_recrawl_file_note = './src/conference/evaluate/tri_all_recrawl_final.csv'

    # 5. Gọi hàm xử lý chính
    process_files_by_note(
        input_files=input_csv_files_for_note_check,
        output_dir=individual_output_directory_note,
        aggregated_correct_path=aggregated_correct_file_note,
        aggregated_recrawl_path=aggregated_recrawl_file_note,
        recrawled_files=recrawled_results_files # TRUYỀN THAM SỐ MỚI VÀO ĐÂY
    )


--- Đang xử lý file input: ./src/conference/evaluate/batch4.csv ---
Đã đọc thành công 47 dòng.
Kết quả tách ban đầu: 20 dòng không có ghi chú, 27 dòng có ghi chú.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch5.csv ---
Đã đọc thành công 48 dòng.
Kết quả tách ban đầu: 19 dòng không có ghi chú, 29 dòng có ghi chú.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch9.csv ---
Đã đọc thành công 45 dòng.
Kết quả tách ban đầu: 21 dòng không có ghi chú, 24 dòng có ghi chú.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý file input: ./src/conference/evaluate/batch14.csv ---
Đã đọc thành công 44 dòng.
Kết quả tách ban đầu: 18 dòng không có ghi chú, 26 dòng có ghi chú.
Đã lưu output riêng lẻ vào thư mục './src/conference/evaluate/tri_check_outputs'

--- Đang xử lý fil

In [15]:
import csv

# --- Cấu hình tên file ---
file_danh_gia = './src/conference/evaluate/individual_outputs/batch12_recrawl.csv'
file_core_2023 = './src/conference/csv/CORE_2023.csv'
output_file = './src/conference/evaluate/file_to_recrawl_batch_12_lan_1.csv'

# --- Bước 1: Đọc file "đánh giá" và tạo một set các khóa (title, acronym) để tra cứu ---
# Sử dụng set để tra cứu nhanh hơn rất nhiều so với list
lookup_keys = set()

try:
    # Mở file với encoding='utf-8' để đảm bảo đọc được tiếng Việt
    with open(file_danh_gia, mode='r', newline='', encoding='utf-8') as f_danhgia:
        # Dùng DictReader để dễ dàng truy cập cột bằng tên header
        reader = csv.DictReader(f_danhgia)
        for row in reader:
            # Lấy title và acronym, loại bỏ khoảng trắng thừa ở đầu và cuối
            title = row.get('title', '').strip()
            acronym = row.get('acronym', '').strip()
            
            # Chỉ thêm vào set nếu cả title và acronym đều có giá trị
            if title and acronym:
                lookup_keys.add((title, acronym))
    print(f"Đã tìm thấy {len(lookup_keys)} khóa (title, acronym) duy nhất từ file '{file_danh_gia}'.")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_danh_gia}'. Vui lòng kiểm tra lại tên và đường dẫn file.")
    exit()


# --- Bước 2: Đọc file CORE_2023, lọc và ghi ra file mới ---
matching_rows = []

try:
    with open(file_core_2023, mode='r', newline='', encoding='utf-8') as f_core:
        # File CORE không có header, dùng csv.reader bình thường
        reader = csv.reader(f_core)
        for row in reader:
            # Đảm bảo dòng có đủ cột để tránh lỗi IndexError
            if len(row) > 2:
                # Cột 2 là title (index 1), Cột 3 là acronym (index 2)
                core_title = row[1].strip()
                core_acronym = row[2].strip()
                
                # Tạo khóa từ file CORE
                current_key = (core_title, core_acronym)
                
                # Kiểm tra xem khóa này có trong set đã tạo ở bước 1 không
                if current_key in lookup_keys:
                    matching_rows.append(row)

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_core_2023}'. Vui lòng kiểm tra lại tên và đường dẫn file.")
    exit()


# --- Bước 3: Ghi các dòng đã lọc vào file output ---
if matching_rows:
    with open(output_file, mode='w', newline='', encoding='utf-8') as f_output:
        writer = csv.writer(f_output)
        writer.writerows(matching_rows)
    print(f"Đã lọc và ghi {len(matching_rows)} conference vào file '{output_file}'.")
else:
    print("Không tìm thấy conference nào trùng khớp.")

Đã tìm thấy 19 khóa (title, acronym) duy nhất từ file './src/conference/evaluate/individual_outputs/batch12_recrawl.csv'.
Đã lọc và ghi 16 conference vào file './src/conference/evaluate/file_to_recrawl_batch_12_lan_1.csv'.
